In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
start_time = time.time()

!mkdir cityscapes
!unzip -qo /content/drive/MyDrive/datasets/leftImg8bit_trainvaltest.zip -d /content/cityscapes
!unzip -qo /content/drive/MyDrive/datasets/gtFine_trainvaltest.zip -d /content/cityscapes

end_time = time.time()
execution_time = end_time - start_time
execution_time = int(execution_time/60)
print("Execution time:", execution_time, "minutes")

Execution time: 4 minutes


In [ ]:
!pip install -q visdom ood_metrics cityscapesscripts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.6/473.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/mcordts/cityscapesScripts.git

Cloning into 'cityscapesScripts'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 648 (delta 184), reused 165 (delta 155), pack-reused 427
Receiving objects: 100% (648/648), 796.28 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (370/370), done.


In [ ]:
import time
start_time = time.time()

import os
os.environ["CITYSCAPES_DATASET"] = "/content/cityscapes"
!python cityscapesScripts/cityscapesscripts/preparation/createTrainIdLabelImgs.py

end_time = time.time()
execution_time = end_time - start_time
execution_time = int(execution_time/60)
print("Execution time:", execution_time, "minutes")

Processing 5000 annotation files
Progress: 100.0 % Execution time: 2 minutes


In [ ]:
!git clone https://github.com/shayanit/anomaly-segmentation-for-road-scenes

Cloning into 'anomaly-segmentation-for-road-scenes'...
remote: Enumerating objects: 763, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 763 (delta 41), reused 66 (delta 26), pack-reused 681
Receiving objects: 100% (763/763), 562.14 MiB | 40.52 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [ ]:
%cd /content/anomaly-segmentation-for-road-scenes/train
from main import MyCoTransformExtension
from dataset import cityscapes

/content/anomaly-segmentation-for-road-scenes/train


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

barlowtwins_resnet = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
for param in barlowtwins_resnet.parameters():
    param.requires_grad = False #FREEZE
barlowtwins_resnet = nn.Sequential(*list(barlowtwins_resnet.children())[:-2])
num_classes = 20
segmentation_head = nn.Conv2d(2048, num_classes, kernel_size=1)
upsample = nn.Upsample(size=(224,224), mode='bilinear', align_corners=False)
BarlowTwinsModel = nn.Sequential(barlowtwins_resnet, segmentation_head, upsample)

Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth
100%|██████████| 90.0M/90.0M [00:00<00:00, 101MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.hub import load as hub_load
import numpy as np


# Load Cityscapes dataset
datadir = '/content/cityscapes'
co_transform = MyCoTransformExtension(False, augment=True)
co_transform_val = MyCoTransformExtension(False, augment=False)
dataset_train = cityscapes(datadir, co_transform, 'train')
dataset_val = cityscapes(datadir, co_transform_val, 'val')

loader = DataLoader(dataset_train, num_workers=2, batch_size=8, shuffle=True)
loader_val = DataLoader(dataset_val, num_workers=2, batch_size=8, shuffle=False)

print(dataset_train[0][0].shape)
print(dataset_train[0][1].shape)

/content/cityscapes/leftImg8bit/train
/content/cityscapes/leftImg8bit/val
torch.Size([3, 224, 224])
torch.Size([1, 224, 224])


#Train


In [ ]:
import torch.optim as optim

import time
start_time = time.time()

# Instantiate the model
model = BarlowTwinsModel

# Define loss function
criterion = nn.CrossEntropyLoss()

# Choose optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs=10
print_every=10
best_model = 0
best_val_loss_avg=float('inf')

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_batches = len(loader)
    for batch_idx, (images, labels) in enumerate(loader):
        if (batch_idx + 1) % print_every == 0 or (batch_idx + 1) == total_batches:
          print(f"Epoch {epoch+1}, batch {batch_idx+1}/{total_batches}")
        optimizer.zero_grad()
        labels = labels.squeeze(1)
        # print(f"images shape: {images.shape}")
        outputs = model(images)
        # print(f"outputs shape: {outputs.shape}")
        labels = labels.long()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Training Loss: {running_loss / len(loader)}")
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    val_total_batches = len(loader_val)

    with torch.no_grad():  # Turn off gradient calculation during validation
        for batch_idx, (val_images, val_labels) in enumerate(loader_val):
            val_labels = val_labels.squeeze(1)
            val_outputs = model(val_images)
            val_labels = val_labels.long()
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()

    val_loss_avg = val_running_loss / len(loader_val)
    print(f"Validation Loss: {val_loss_avg}")
    torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, f"/content/drive/MyDrive/datasets/barlowtwins-frozen-{epoch+1}.pth")
    if (val_loss_avg<best_val_loss_avg):
      best_val_loss_avg=val_loss_avg
      best_model=epoch+1
    print(f"Best model is: {best_model}")

end_time = time.time()
execution_time = end_time - start_time
execution_time = int(execution_time/60)
print("Execution time:", execution_time, "minutes")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, batch 10/372
Epoch 1, batch 20/372
Epoch 1, batch 30/372
Epoch 1, batch 40/372
Epoch 1, batch 50/372
Epoch 1, batch 60/372
Epoch 1, batch 70/372
Epoch 1, batch 80/372
Epoch 1, batch 90/372
Epoch 1, batch 100/372
Epoch 1, batch 110/372
Epoch 1, batch 120/372
Epoch 1, batch 130/372
Epoch 1, batch 140/372
Epoch 1, batch 150/372
Epoch 1, batch 160/372
Epoch 1, batch 170/372
Epoch 1, batch 180/372
Epoch 1, batch 190/372
Epoch 1, batch 200/372
Epoch 1, batch 210/372
Epoch 1, batch 220/372
Epoch 1, batch 230/372
Epoch 1, batch 240/372
Epoch 1, batch 250/372
Epoch 1, batch 260/372
Epoch 1, batch 270/372
Epoch 1, batch 280/372
Epoch 1, batch 290/372
Epoch 1, batch 300/372
Epoch 1, batch 310/372
Epoch 1, batch 320/372
Epoch 1, batch 330/372
Epoch 1, batch 340/372
Epoch 1, batch 350/372
Epoch 1, batch 360/372
Epoch 1, batch 370/372
Epoch 1, batch 372/372
Epoch 1, Training Loss: 2.040247703431755
Validation Loss: 1.679991152551439
Best model is: 0
Epoch 2, batch 10/372
Epoch 2, batch 20/3